<h1> Q3: Real World Dataset</h1>

In [2]:
import os
import autograd.numpy as np  # when testing gradient
from cvxopt import matrix, solvers
import pandas as pd



print("Testing")

Testing


In [3]:
##imports from q1


def l2_Loss(w, X,y):

    predicted_diff = (np.linalg.norm(X @ w - y)**2)
    relative_result = predicted_diff / (2*X.shape[0])
    return relative_result


def l1_Loss(w, X,y):
    predicted = np.abs(X@w - y)
    avg = np.mean(predicted)
    return avg

def lInf_Loss(w, X,y):
    diff = np.abs(X@w-y)
    max_val = np.amax(diff)
    return max_val


def compute_loss(w_l2, w_l1,w_Linf,X,y):
    results = np.zeros([3, 3])

    results[0,0] = l2_Loss(w_l2, X,y)
    results[1,0] = l2_Loss(w_l1,X,y)
    results[2,0] = l2_Loss(w_Linf, X, y)
    
    results[0,1] = l1_Loss(w_l2, X,y)
    results[1,1] = l1_Loss(w_l1,X,y)
    results[2,1] = l1_Loss(w_Linf, X, y)

    results[0,2] = lInf_Loss(w_l2, X,y)
    results[1,2] = lInf_Loss(w_l1,X,y)
    results[2,2] = lInf_Loss(w_Linf, X, y)

    return results

In [6]:
#q2  imports

#Q2c

def sigmoid(x):
    return np.exp(-np.logaddexp(0, -x))


def logisticRegObj(w, X, y):
    pred = X @ w
    n = X.shape[0]
    
    objval = (1/n) * np.sum(np.logaddexp(0, -pred) - y * pred)
    grad = (1/n) * X.T @ (sigmoid(pred) - y)
    
    return objval, grad


In [7]:
#Q3a)

def preprocessAutoMPG(dataset_folder):

    data_df = pd.read_csv(os.path.join(dataset_folder, "auto-mpg.data"), 
                          header=None, 
                          delim_whitespace=True)
    
    print(data_df.head())


    del data_df[8]    #car name
    del data_df[7]  #origin

    df_data = data_df.dropna() #drop rows with any missing data

    labels = df_data[0].to_numpy(float)[:, None]
    del df_data[0]

    matrix = df_data.to_numpy(float)

    return matrix, labels





def gd(func, w_init, X, y, step_size, max_iter, tol=1e-10):

    w = w_init

    for i in range(max_iter):
        objval, gradient = func(w,X,y)
        
        norm = np.linalg.norm(gradient)
        if norm < tol: #stop when gradient is good enough
            break

        w = w - step_size * gradient

    return 2



In [5]:
#Q3b

def runAutoMPG(dataset_folder):

    X, y = preprocessAutoMPG(dataset_folder)
    n, d = X.shape
    X = np.concatenate((np.ones((n, 1)), X), axis=1) # augment
    n_runs = 100
    train_loss = np.zeros([n_runs, 3, 3]) # n_runs * n_models * n_metrics
    test_loss = np.zeros([n_runs, 3, 3]) # n_runs * n_models * n_metrics
    
    for r in range(n_runs):
        
        #partition data
        rand_indices = np.random.permutation(n)
        num_training = n // 2
        train_indices = rand_indices[:num_training]
        test_indices = rand_indices[num_training:]

        X_train, y_train = X[train_indices], y[train_indices]
        X_test, y_test = X[test_indices], y[test_indices]

        l1model = l1_Loss(X_train,y_train)
        l2model = l2_loss(X_train,y_train)
        linfmodel = linfloss(X_train,y_train)

        train_loss = compute_loss(l2model,l1model,linfmodel,X_train,y_train)
        test_loss = compute_loss(l2model,l1model,linfmodel,X_test,y_test)

        avg_train_losses = np.mean(train_loss, axis=0)
        avg_test_losses = np.mean(test_loss, axis=0)


        return avg_train_losses, avg_test_losses

In [7]:
#Q3c

def preprocessSonar(dataset_folder):

    data_df = pd.read_csv(os.path.join(dataset_folder, "sonar.all-data"), 
                          header=None, 
                          delim_whitespace=True)


    # Convert labels
    data_df[60] = (data_df[60] == 'R').astype(float)
    y = data_df[60].to_numpy()[:, None]
    
    #remove labels from the data
    del data_df[60]
    X = data_df.to_numpy()

    return X, y

sonar_datapath = "/Users/mariopardo/OnThisMac/Programming/C3105_Group62/Assignment1/Data/connectionist+bench+sonar+mines+vs+rocks"

preprocessSonar(sonar_datapath)

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60],
      dtype='int64')


In [8]:
#q3d

def runSonar(dataset_folder):

    X, y = preprocessSonar(dataset_folder)
    n, d = X.shape
    X = np.concatenate((np.ones((n, 1)), X), axis=1) # augment
    eta_list = [0.1, 1, 10, 100]

    train_acc = np.zeros([len(eta_list)])
    val_acc = np.zeros([len(eta_list)])
    test_acc = np.zeros([len(eta_list)])

    indices = np.random.permutation(n)
    train_size = round(n * 0.4)
    val_size = round(n * 0.4)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    test_indices = indices[train_size + val_size:]

    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xval, yval = X[val_indices], y[val_indices]
    Xtest, ytest = X[test_indices], y[test_indices]


    for i, eta in enumerate(eta_list):
        w_init = np.zeros([d + 1, 1])
        w = gd(logisticRegObj, w_init, Xtrain, ytrain, eta, max_iter=1000, tol=1e-8)

        # TODO: Evaluate the model's accuracy on the training
        # data. Save it to `train_acc`
        train_pred = (1/2) * (1 * np.sign(Xtrain @ w))
        train_acc[i] = np.sum(ytrain == train_pred) / Xtrain.shape[0]

        # TODO: Evaluate the model's accuracy on the validation
            #       data. Save it to `val_acc`
        val_pred = 0.5 * (1 + np.sign(Xval @ w))
        val_acc[i] = np.sum(yval == val_pred) / Xval.shape[0]

            
        test_pred = 0.5 * (1 + np.sign(Xtest @ w))
        test_acc[i] = np.sum(ytest == test_pred) / Xtest.shape[0]

    return train_acc, val_acc, test_acc